In [6]:
!pip install gradio_client

In [7]:
import torch
from transformers import pipeline
import gradio as gr
import time

In [8]:
pipe= pipeline(task = "text-generation", model = "TinyLlama/TinyLlama-1.1B-Chat-v1.0", dtype=torch.bfloat16, device_map="auto")

Device set to use cuda:0


In [9]:
def chat(user_input):
    # We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
    start_time = time.time()

    messages = [
        {
            "role": "system",
            "content": "write a blog on give topic",
},
        {"role": "user", "content": user_input},
    ]

    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.5, top_k=50, top_p=0.95)
    raw_output = outputs[0]["generated_text"]
    assistant_response = raw_output.split("<|assistant|>")[-1].strip()

    end_time = time.time()
    latency = end_time - start_time

    response = f"📝 **Response:**\n{assistant_response}\n\n⏱️ **Latency:** {latency:.2f} seconds"
    return response

# GPU Usage:
    # model parameter = 1.1B
    # pricision = bfloat16 = 2 bytes
    # total model size = 2.2 GB
    # for activation, over head only small GPU RAM usage will taken in MBs

In [10]:
# Gradio interface
iface = gr.Interface(
    fn=chat,
    inputs=gr.Textbox(lines=2, placeholder="Say a topic to write blog about"),
    outputs=gr.Textbox(lines=5, label="Chatbot Response"),
    title="Chatbot"
)

iface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://447eee3f6ad2c5b60d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
